### Install requirements

In [3]:
%pip show duckdb

Name: duckdb
Version: 0.10.3
Summary: DuckDB in-process database
Home-page: https://www.duckdb.org
Author: 
Author-email: 
License: MIT
Location: /Users/sam/Library/Python/3.9/lib/python/site-packages
Requires: 
Required-by: duckdb_engine, llama-index-vector-stores-duckdb
Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 4.6 MB/s eta 0:00:0000:0100:01
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion c

### Load Libraries, Set Global Variables and Settings

In [20]:
from __future__ import print_function
import logging
import sys
import os
import pandas as pd
import glob
from dotenv import load_dotenv
import logging
import sys
import duckdb
from googleFunctions import Form
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ( Settings, VectorStoreIndex, SimpleDirectoryReader)
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import (StorageContext, ServiceContext)
from llama_index.core.node_parser import MarkdownNodeParser
import chromadb
import datetime
import numpy as np
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter
from sqlalchemy import *
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import SQLDatabase, Document
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.query_engine import SQLJoinQueryEngine
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser
from sqlalchemy import (create_engine,MetaData,Table,Column,String,Integer,select,column,)
from sqlalchemy.dialects.postgresql import (INTEGER, FLOAT, BIGINT, VARCHAR, DOUBLE_PRECISION)
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (SQLTableNodeMapping,ObjectIndex,SQLTableSchema,)
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import SQLAutoVectorQueryEngine


# NOTE: This is ONLY necessary in jupyter notebook.
import nest_asyncio

nest_asyncio.apply()

logging.getLogger().setLevel(logging.ERROR)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

load_dotenv('../Credentials/.env')

#Azure OpenAI Creds
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
credential = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_api_version = "2024-04-01-preview"
azure_openai_embedding_deployment = "text-embedding-ada-002"
embedding_model_name = "text-embedding-ada-002"
llm_model_name = "gpt-35-turbo-16k"
api_type = "azure"

#Google Creds
CREDENTIALS_FILE = '../Credentials/credentials.json'
CLIENT_SECRETS_FILE = '../Credentials/client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/forms.body']
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

user_path = "Users/sam/dev/spaghetti_dev/"

llm = AzureOpenAI(
            model = llm_model_name,
            deployment_name = llm_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type
        )

embed_model = AzureOpenAIEmbedding(
            model = embedding_model_name,
            deployment_name = embedding_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type,
            embed_batch_size=50
        )

Settings.llm = llm
Settings.embed_model = embed_model

### Connect to ChromaDB, Create Collection

In [23]:
remote_db = chromadb.HttpClient(host='localhost',port=8000)
remote_db.delete_collection("quickstart")
chroma_collection = remote_db.get_or_create_collection("quickstart")

### Load PDF Course Files, Add Metadata, Create Vector Index

In [26]:
logging.getLogger().setLevel(logging.WARNING)
pd.set_option('future.no_silent_downcasting', True)
# embed that content, with metadata for where they came from/what consolidation exercise they're a part of 

reader = SimpleDirectoryReader("/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/studentguides/", recursive=True, filename_as_id=True, required_exts=[".pdf", ".docx", ".xlsx", ".pptx"])

documents = []
for docs in reader.iter_data():
    file_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_file.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    file_df = pd.read_json(file_filename)
    file_metadata = file_df.loc[file_df['filename'] == docs[0].metadata['file_name'][7:].replace('_','+')]
    if file_metadata.empty != True:                
        file_metadata = file_metadata.squeeze().to_dict()
        file_metadata = pd.DataFrame(file_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
        file_metadata = file_metadata.to_dict('records')[0]
        folder = file_metadata.get('folder_id')
    else:
        file_metadata = {}
        folder = ''

    folder_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_folder.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    folder_df = pd.read_json(folder_filename)
    folder_metadata = folder_df.loc[folder_df['id'] == folder]
    if folder_metadata.empty != True:
        folder_metadata = folder_metadata.squeeze().to_dict()
        folder_metadata = pd.DataFrame(folder_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
        folder_metadata = folder_metadata.to_dict('records')[0]
        if 'Week' in folder_metadata['full_name']:
            week = [i for i in folder_metadata['full_name'].split("/") if 'Week' in i][0].replace('Week','').replace(' ','')
            folder_metadata.update({"week":week})
        full_name = folder_metadata['full_name'].split("/")[-1]
        folder_metadata.update({"folder_name":full_name})
    else:
        folder_metadata = {}
    
    course_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_course.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    course_df = pd.read_json(course_filename)        
    course_metadata = course_df.loc[course_df['id'] == folder_metadata.get('context_id')]
    if course_metadata.empty != True:
        course_metadata = course_metadata.squeeze().to_dict()
        course_id = folder_metadata.get('context_id')
    else:
        course_metadata = {}
            
    for doc in docs:
        doc.metadata.update({"file_id": file_metadata.get('id'), "folder_id":file_metadata.get('folder_id'), "display_name":file_metadata.get('display_name')})
        doc.metadata.update({"week": folder_metadata.get('week'),  "folder_name": folder_metadata.get('folder_name')})
        doc.metadata.update({"course_id": course_metadata.get('id'), "course_name":course_metadata.get('name'),"course_code":course_metadata.get('course_code'),"course_term":course_metadata.get('term', {}).get('name')}) 
    documents.extend(docs)


parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
))

nodes = parser.get_nodes_from_documents(documents)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection, nodes=nodes)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, llm=llm
)

**********
Trace: index_construction
    |_embedding -> 1.004166 seconds
    |_embedding -> 1.107037 seconds
    |_embedding -> 0.767102 seconds
    |_embedding -> 0.74843 seconds
    |_embedding -> 0.945666 seconds
    |_embedding -> 0.734158 seconds
    |_embedding -> 0.852884 seconds
    |_embedding -> 0.843995 seconds
    |_embedding -> 0.780787 seconds
    |_embedding -> 0.719542 seconds
    |_embedding -> 0.82279 seconds
    |_embedding -> 0.763947 seconds
    |_embedding -> 0.759422 seconds
    |_embedding -> 0.829662 seconds
    |_embedding -> 0.856199 seconds
    |_embedding -> 0.754701 seconds
    |_embedding -> 0.901613 seconds
    |_embedding -> 0.763049 seconds
    |_embedding -> 0.759382 seconds
    |_embedding -> 0.667142 seconds
**********


### Check Metadata

In [27]:
doc.metadata

{'page_label': '3',
 'file_name': '130681_updated_FA10_Student_Guide.pdf',
 'file_path': '/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/studentguides/130681_updated_FA10_Student_Guide.pdf',
 'file_type': 'application/pdf',
 'file_size': 469164,
 'creation_date': '2024-05-21',
 'last_modified_date': '2024-05-21',
 'file_id': 18357657,
 'folder_id': 3679135,
 'display_name': 'updated FA10 Student Guide.pdf',
 'week': '7',
 'folder_name': 'FA 10',
 'course_id': 130681,
 'course_name': 'PWY 100: Foundations - HMS - 08/07/2023 - 11/03/2023',
 'course_code': 'PWY 100',
 'course_term': '2023-2024 Fall'}

### Define RetrieverQueryEngine

In [28]:
vector_store_info = VectorStoreInfo(
    content_info="Student guides to help prepare for consolidation assessments",
    metadata_info=[
        MetadataInfo(
            name="page_label",
            description="What page of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_name",
            description="The name of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_path",
            description="The file path of the context file",
            type="string",
        ),
        MetadataInfo(
            name="file_type",
            description="The type of file",
            type="string",
        ),
        MetadataInfo(
            name="file_size",
            description="The size of the file in bytes",
            type="integer",
        ),
        MetadataInfo(
            name="creation_date",
            description="When the file was created",
            type="string",
        ),
        MetadataInfo(
            name="last_modified_date",
            description="When the file was last modified",
            type="string",
        ),
        MetadataInfo(
            name="display_name",
            description="The name of the file",
            type="string",
        ),
        MetadataInfo(
            name="week",
            description="The week the context was administered",
            type="string",
        ),
        MetadataInfo(
            name="folder_name",
            description="The course folder that contains the file",
            type="string",
        ),
        MetadataInfo(
            name="course_id",
            description="The unique identifier of the course",
            type="integer",
        ),
        MetadataInfo(
            name="course_name",
            description="The full name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_code",
            description="The shortened name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_term",
            description="What term the course was offered in",
            type="string",
        ),
    ],
)

vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=llm
)

### Set Up Query Engine Tools

In [29]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table svr_course_quiz_view, containing columns:"
        " quiz_id, quiz_type, quiz_title, history_id, submission_id,"
        " student_score, quiz_question_count, quiz_points_possible, question_points_possible," 
        "answer_points_scored, attempt, question_name, question_type, question_text, question_answer, and student_answer"
    ),
)


vector_tool = QueryEngineTool.from_defaults(
        query_engine=retriever_query_engine,
        description=f"Useful for answering semantic questions about consolidation assessments, and general course-related questions like when certain material is being taught",
    )


### Define Query Engine

In [30]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
    llm=llm
)

### Query Test 1: Ability to identify topics covered in a specific assessment

In [13]:
response = query_engine.query("What material is covered in CBB 3 Consolidation Assessment Exercise?")

response.response

Querying other query engine: The question is asking about the material covered in a specific assessment, which is related to consolidation assessments and general course-related questions.
Query Engine response: The material covered in the CBB 3 Consolidation Assessment Exercise includes proteolysis and protein degradation.
**********
Trace: query
**********


'The material covered in the CBB 3 Consolidation Assessment Exercise includes proteolysis and protein degradation.'

### Query Test 2: Ability to query SQL db, synthesize results and deliver a cohesive answer

In [13]:
response = query_engine.query("Compare all student answers to the question answer for question 1 of CBB 1 Consolidation Assessment Exercise. What concept did students best understand? Which concept was most frequently not mentioned?")

response.response += ' '
out = response.response.split('. ')[:-1]
print('.\n'.join(out))

Querying SQL database: This choice mentions the table svr_course_quiz_view which contains the necessary columns to compare student answers to the question answer.
SQL query: SELECT question_name, question_text, question_answer, student_answer
FROM svr_course_quiz_view
WHERE quiz_title = 'CBB 1 Consolidation Assessment Exercise'
AND question_name = 'Question 1'
SQL response: The concept that students best understood from their answers to question 1 of the CBB 1 Consolidation Assessment Exercise is the interpretation of a hemoglobin-oxygen dissociation curve. Many students correctly explained that the curve shows the percent saturation or oxygen content of the blood as a function of oxygen tension (PO2). They mentioned that the curve represents the relationship between the partial pressure of oxygen and the fraction of hemoglobin that is bound to oxygen. They also discussed the significance of the P50 value, which represents the partial pressure of oxygen at which hemoglobin is 50% satur

### Query Test 3: Ability to identify material from specific time, create quiz questions, create Google quiz

In [14]:

response = query_engine.query(
    'Identify what material is taught in week 7. Then create a 5 question review quiz on the material taught in week 7. Do not ask questions about text from a photo/graph. Do not ask any questions about textbook sections. Do not ask any questions about learning objectives. Every question should have only one correct answer. Every question should have 4 potential answers. Format the output like this:{"info":{"title": "", "description": "", "questions": [ { "question": "","type": "RADIO","options": [], "correct_answer": ""}]}}'
)

# Google Quiz Creation
form = Form(file_type='credentials', loginfile=CREDENTIALS_FILE,
        discovery_doc=DISCOVERY_DOC, scopes=SCOPES, sentence_response = response)


Querying other query engine: Useful for answering semantic questions about consolidation assessments, and general course-related questions like when certain material is being taught
Query Engine response: {"info":{"title": "Week 7 Review Quiz", "description": "This quiz is designed to test your knowledge on the material taught in week 7. Each question has four potential answers, but only one is correct. Good luck!", "questions": [
{ "question": "Which of the following is NOT a function of the mesentery, ligaments, and omenta?", "type": "RADIO", "options": ["A. Provide structural support", "B. Store fat", "C. Connect organs to the abdominal wall", "D. Create a general vascular map"], "correct_answer": "D. Create a general vascular map"},
{ "question": "What is the term used to describe the organs and structures of the abdomen that are located outside of the peritoneal cavity?", "type": "RADIO", "options": ["A. Intraperitoneal", "B. Extraperitoneal", "C. Retroperitoneal", "D. Secondarily

### Chat Session Testing

In [35]:
# response = query_engine.query(
#     'Identify what material is taught in week 7. Only respond with the topics, separated with commas.'
# )

# response.response

response = query_engine.query(
    'Create a 6 question review quiz on the hormonal Control of Metabolism. Do not ask questions about text from a photo/graph. Do not ask any questions about textbook sections. Do not ask any questions about learning objectives. Every question should have only one correct answer. Every question should have 4 potential answers. 2 of the questions should be easy in difficulty. 2 of the questions should be medium in difficulty. 2 of the questions should be hard in difficulty. Format the output like this:{"info":{"title": "", "description": "", "questions": [ { "question": "","type": "RADIO","options": [], "correct_answer": ""}]}}'
)

# Google Quiz Creation
form = Form(file_type='credentials', loginfile=CREDENTIALS_FILE,
        discovery_doc=DISCOVERY_DOC, scopes=SCOPES, sentence_response = response)

Querying SQL database: This choice is relevant because it mentions creating a quiz and provides information about the columns in the table svr_course_quiz_view, which can be used to generate the quiz.
SQL query: INSERT INTO svr_course_quiz_view (quiz_type, quiz_title, quiz_question_count, quiz_points_possible)
VALUES ('review', 'Hormonal Control of Metabolism', 6, 6);
SELECT * FROM svr_course_quiz_view WHERE quiz_title = 'Hormonal Control of Metabolism';
SQL response: {"info":{"title": "Hormonal Control of Metabolism Review Quiz", "description": "", "questions": [ { "question": "What is the primary hormone responsible for regulating metabolism?", "type": "RADIO","options": ["Insulin", "Glucagon", "Thyroid hormone", "Cortisol"], "correct_answer": "Insulin"}, { "question": "Which hormone stimulates the breakdown of glycogen into glucose?", "type": "RADIO","options": ["Insulin", "Glucagon", "Thyroid hormone", "Cortisol"], "correct_answer": "Glucagon"}, { "question": "What is the main func